# Model Comparison for Indonesian E-commerce Review Summarization

This notebook compares different instruction-tuned LLMs for summarizing Indonesian e-commerce reviews.

## Setup

In [ ]:
import sys
sys.path.append('../src')

from indo_ecommerce_review_summarization.preprocessing import clean_text
from indo_ecommerce_review_summarization.models import create_summarization_prompt
from indo_ecommerce_review_summarization.evaluation import evaluate_predictions, format_rouge_scores

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Sample Data

Let's prepare some sample reviews and reference summaries for comparison.

In [ ]:
# Sample reviews and reference summaries
test_data = [
    {
        "reviews": [
            "Barang bagus banget, sesuai deskripsi. Pengiriman cepat, packing rapi. Seller responsif."
        ],
        "reference": "Produk berkualitas dengan pengiriman cepat dan seller responsif."
    },
    {
        "reviews": [
            "Kualitas produk oke sih, tapi pengiriman agak lama. Overall masih puas."
        ],
        "reference": "Produk bagus namun pengiriman lambat."
    },
    {
        "reviews": [
            "Harga murah, kualitas lumayan. Recommended buat budget terbatas."
        ],
        "reference": "Harga terjangkau dengan kualitas yang baik."
    }
]

print(f"Loaded {len(test_data)} test examples")

## Model 1: Mistral-7B-Instruct

In [ ]:
from indo_ecommerce_review_summarization.models import load_model

# Load Mistral model
mistral_model = load_model(
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    model_type="huggingface",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

print("Mistral model loaded!")

In [ ]:
# Generate summaries with Mistral
mistral_predictions = []

for example in test_data:
    prompt = create_summarization_prompt(
        reviews=example["reviews"],
        model_type="mistral",
        max_length=30
    )
    
    summary = mistral_model.generate(
        prompt,
        max_new_tokens=100,
        temperature=0.7
    )
    
    mistral_predictions.append(summary)
    print(f"Review: {example['reviews'][0]}")
    print(f"Summary: {summary}")
    print("-" * 80)

## Evaluation

Compare model performance using ROUGE metrics.

In [ ]:
# Extract references
references = [example["reference"] for example in test_data]

# Evaluate Mistral
mistral_scores = evaluate_predictions(
    predictions=mistral_predictions,
    references=references
)

print("Mistral-7B-Instruct Performance:")
print(format_rouge_scores(mistral_scores))

## Alternative: Using Other Models

You can easily swap in other models by changing the model name and prompt template.

### Example with LLaMA

In [ ]:
# Uncomment to use LLaMA model
# llama_model = load_model(
#     model_name="meta-llama/Llama-2-7b-chat-hf",
#     model_type="huggingface",
#     load_in_4bit=True,
#     torch_dtype=torch.float16
# )

# llama_predictions = []
# for example in test_data:
#     prompt = create_summarization_prompt(
#         reviews=example["reviews"],
#         model_type="llama",
#         max_length=30
#     )
#     summary = llama_model.generate(prompt, max_new_tokens=100)
#     llama_predictions.append(summary)

# llama_scores = evaluate_predictions(llama_predictions, references)
# print("LLaMA-2-7B Performance:")
# print(format_rouge_scores(llama_scores))

## Comparison Summary

In [ ]:
import pandas as pd

# Create comparison dataframe
comparison_data = {
    "Model": ["Mistral-7B-Instruct"],
    "ROUGE-1 F1": [mistral_scores["rouge1"]["fmeasure"]],
    "ROUGE-2 F1": [mistral_scores["rouge2"]["fmeasure"]],
    "ROUGE-L F1": [mistral_scores["rougeL"]["fmeasure"]],
}

# Uncomment to add more models
# comparison_data["Model"].append("LLaMA-2-7B")
# comparison_data["ROUGE-1 F1"].append(llama_scores["rouge1"]["fmeasure"])
# comparison_data["ROUGE-2 F1"].append(llama_scores["rouge2"]["fmeasure"])
# comparison_data["ROUGE-L F1"].append(llama_scores["rougeL"]["fmeasure"])

df = pd.DataFrame(comparison_data)
print("\nModel Comparison:")
print(df.to_string(index=False))

## Conclusion

This notebook demonstrated:
1. How to compare multiple LLMs for Indonesian review summarization
2. Evaluation using ROUGE metrics
3. A framework that's easy to extend to other models

## Recommendations

- Test with larger datasets for more reliable comparisons
- Experiment with different prompt templates
- Consider human evaluation for quality assessment
- Fine-tune models on Indonesian e-commerce data for better performance